In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,ReLU,TimeDistributed,Embedding,LSTM,RepeatVector

In [ ]:
df=pd.read_csv("Q4 MT Eng Tel Dataset.txt",delimiter="\t")
df.columns=['English','Telugu','Metadata']

In [ ]:
df.columns

In [ ]:
df

In [ ]:
english=df['English'].tolist()
telugu=df['Telugu'].tolist()
eng_tokenizer=Tokenizer()
tel_tokenizer=Tokenizer()
eng_tokenizer.fit_on_texts(english)
tel_tokenizer.fit_on_texts(telugu)
eng_len=len(eng_tokenizer.word_index)+1
tel_len=len(tel_tokenizer.word_index)+1
eng_seq=eng_tokenizer.texts_to_sequences(english)
tel_seq=tel_tokenizer.texts_to_sequences(telugu)
inp_eng=pad_sequences(eng_seq,padding="post")
inp_tel=pad_sequences(tel_seq,padding="post")
model=Sequential([
    Embedding(eng_len,256,input_length=inp_eng.shape[1]),
    LSTM(512,return_sequences=False),
    RepeatVector(inp_tel.shape[1]),
    LSTM(512,return_sequences=True),
    TimeDistributed(Dense(tel_len,activation="softmax"))
])
print(inp_eng.shape,inp_tel.shape)

In [ ]:

model.compile(optimizer="adam",loss="sparse_categorical_crossentropy",metrics=["accuracy"])
model.fit(inp_eng,np.expand_dims(inp_tel,-1),epochs=20)

In [ ]:
def translate_sentence(sentence):
    seq = eng_tokenizer.texts_to_sequences([sentence])
    padded = pad_sequences(seq, maxlen=inp_eng.shape[1], padding='post')
    prediction = np.argmax(model.predict(padded), axis=-1)
    return ' '.join([tel_tokenizer.index_word[i] for i in prediction[0] if i>0])
 # Translate a sentence
input_sentence = input("Enter an English sentence: ")
print(f"Translated sentence: {translate_sentence(input_sentence)}")